In [42]:
import pandas_datareader.data as web 
import datetime 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style

style.use('ggplot')
import yfinance as yf

Apple: AAPL
Microsoft: MSFT
Amazon: AMZN
Tesla: TSLA
Google: GOOGL


In [43]:
# Kullanıcıdan sembol, başlangıç tarihi ve bitiş tarihi almak için örnek bir form
start_date = input("Başlangıç tarihi 1 (yyyy-mm-dd): ")
end_date = input("Bitiş tarihi 1 (yyyy-mm-dd): ")

symbol1 = input("Hisse senedi sembolü 1: ")
symbol2 = input("Hisse senedi sembolü 2: ")
symbol3 = input("Hisse senedi sembolü 3: ")
symbol4 = input("Hisse senedi sembolü 4: ")
symbol5 = input("Hisse senedi sembolü 5: ")

In [44]:
# Tarihleri kontrol edin ve hataları yakalayın
import datetime

def validate_date(date_text):
    try:
        datetime.datetime.strptime(date_text, '%Y-%m-%d')
    except ValueError:
        raise ValueError("Tarih formatı hatalı, yyyy-mm-dd formatında giriniz.")

# Sembole ve tarihlere ilişkin girişleri kontrol edin
if len(symbol1) != 4:
    raise ValueError("Hisse senedi sembolü 1 geçersiz, lütfen 4 karakterli bir sembol giriniz.")
validate_date(start_date)
validate_date(end_date)
if end_date < start_date:
    raise ValueError("Bitiş tarihi başlangıç tarihinden önce olamaz.")

# Aynı kontrol işlemleri 2-5 semboller için de yapılabilir.


In [45]:
import yfinance as yf

symbol_list = [symbol1, symbol2, symbol3, symbol4, symbol5]

stock_data = pd.DataFrame()

for symbol in symbol_list:
    data = yf.download(symbol, start=start_date, end=end_date)
    adj_close = data['Adj Close'].to_frame().rename(columns={'Adj Close': symbol})
    stock_data = pd.concat([stock_data, adj_close], axis=1)



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [46]:
symbol_weights = {}
for i in range(5):
    symbol = input("Enter symbol " + str(i+1) + ": ")
    weight = float(input("Enter weight for " + symbol + ": "))
    symbol_weights[symbol] = weight

In [47]:
# Get daily close prices for each stock in the portfolio
portfolio_prices = pd.DataFrame()
for symbol, weight in symbol_weights.items():
    stock_prices = stock_data[symbol]
    portfolio_prices[symbol] = stock_prices * weight

# Calculate daily returns for the portfolio
portfolio_returns = portfolio_prices.pct_change().dropna()

# Calculate portfolio return and risk
portfolio_return = portfolio_returns.sum(axis=1).mean() * 252
portfolio_risk = portfolio_returns.std(axis=1).mean() * np.sqrt(252)



In [48]:
import scipy.optimize as sco
import pandas as pd

# Convert portfolio_returns to a DataFrame
portfolio_returns = pd.DataFrame(portfolio_returns)

# Calculate the correlation matrix
correlation_matrix = portfolio_returns.corr()

# Define risk tolerance and return target
risk_tolerance = 0.1
return_target = 0.2

# Define optimization function to minimize portfolio risk
def calculate_portfolio_risk(weights):
    portfolio_variance = np.dot(weights.T, np.dot(correlation_matrix, weights))
    portfolio_volatility = np.sqrt(portfolio_variance)
    return portfolio_volatility

def calculate_portfolio_return(weights):
    portfolio_return = np.dot(portfolio_returns.mean().values, weights * 252)
    return -portfolio_return

# Define optimization constraints
constraints = [{'type': 'eq', 'fun': lambda x: np.sum(x) - 1},
               {'type': 'ineq', 'fun': lambda x: calculate_portfolio_return(x) - return_target},
               {'type': 'ineq', 'fun': lambda x: risk_tolerance - calculate_portfolio_risk(x)},
               {'type': 'ineq', 'fun': lambda x: x}]

# Define initial weights
weights = np.ones(len(symbol_weights)) / len(symbol_weights)

# Perform optimization
result = sco.minimize(calculate_portfolio_risk, weights, method='SLSQP', constraints=constraints)

# Print optimized portfolio weights
optimized_weights = result.x.round(2)
for symbol, weight in zip(symbol_weights.keys(), optimized_weights):
    print(f"{symbol}: {weight}")


AAPL: 0.25
MSFT: 0.14
AMZN: 0.21
TSLA: 0.19
GOOGL: 0.21
